In [ ]:
#####################################################################
#
#       aiplatform.CustomTrainingJob -> script_path
#
#####################################################################

In [ ]:
#####################################################################
#
# define the training script
#
#####################################################################

In [ ]:
%%writefile training_script.py

import os
import pickle
import argparse
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from google.cloud import storage

# parse args
parser = argparse.ArgumentParser()
parser.add_argument('--data_bucket_name', dest='data_bucket_name', default="", type=str, help='my gcs bucket with all training data -> w/o gs://')
parser.add_argument('--train_ds', dest='train_ds', default="", type=str, help='CSV file name of the training data')
args = parser.parse_args()

## set params - data
DATA_BUCKET_NAME = args.data_bucket_name
DATA_BUCKET_ROOT = f"/gcs/{DATA_BUCKET_NAME}"
DS_TRAIN_FILENAME = args.train_ds
DS_TRAIN_PATH = f"{DATA_BUCKET_ROOT}/{DS_TRAIN_FILENAME}"

## set params - model
GCS_MODEL_SAVE_PATH = os.environ["AIP_MODEL_DIR"]

# clients
storage_client = storage.Client()

#####################################################################
#
#   BEGIN work
#
#####################################################################

# load data from GCS using GCS fuse
df = pd.read_csv(DS_TRAIN_PATH)
labels = df.pop("label").tolist()
data = df.values.tolist()

# fit the model
skmodel = GradientBoostingClassifier(n_estimators = 100, max_depth = 10, min_samples_split = 100, min_samples_leaf = 100)
skmodel.fit(data, labels)

# save model to GCS
model_filename = "model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(skmodel, model_file)

storage_path = os.path.join(GCS_MODEL_SAVE_PATH, model_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
blob.upload_from_filename(model_filename)

In [ ]:
#####################################################################
#
# kick off the custom training job
#
#####################################################################

In [ ]:
from google.cloud import aiplatform
from datetime import datetime
import os
import pandas as pd

In [ ]:
# specify parameters
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
P = ! gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
PROJECT_NUMBER = P[0]
REGION = "us-central1"
SERVICE_ACCOUNT = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com"

# exercise details
USE_CASE = "CustomTrainingJob"
ML_FRAMEWORK = "scikit"
MODEL_TYPE = "binclass"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

#-------------------
# GCS
#-------------------

# training source data bucket
GCS_DATA_SOURCE_BUCKET_NAME = f"bkt-{REGION}-data"
GCS_DATA_SOURCE_BUCKET_PATH = f"gs://{GCS_DATA_SOURCE_BUCKET_NAME}"

# training data table filename
TRAIN_DS = "tab_class_10inps_1krows_tes_3498.csv"

# use case staging bucket
GCS_BUCKET_NAME = f"bkt-{REGION}-{USE_CASE.lower()}"
GCS_BUCKET_PATH = f"gs://{GCS_BUCKET_NAME}"

In [ ]:
# create use case gcs bucket if needed
! gsutil mb -p {PROJECT_ID} -c standard -l {REGION} {GCS_BUCKET_PATH}
! gsutil ls -L -b {GCS_BUCKET_PATH}

In [ ]:
#-------------------------------------------------------
# JOB SPECIFIC DETAILS
#-------------------------------------------------------

# MODEL - training and serving containers
TRAIN_VERSION  = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "sklearn-cpu.0-23"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

# naming
JOB_NAME = "CustomTrainingJob"
MODEL_DISPLAY_NAME = f"model_{MODEL_TYPE}_{ML_FRAMEWORK}"

# vars
CMDARGS = [f"--data_bucket_name={GCS_DATA_SOURCE_BUCKET_NAME}", f"--train_ds={TRAIN_DS}"]

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

job = aiplatform.CustomTrainingJob(
    display_name = JOB_NAME
    , project = PROJECT_ID
    , location = REGION
    , staging_bucket = GCS_BUCKET_PATH
    , script_path = "training_script.py"
    , container_uri = TRAIN_IMAGE
    , requirements = ["gcsfs"]
    , model_serving_container_image_uri = DEPLOY_IMAGE
    , model_description = "My scikit model description"
)


# Start the training
model = job.run(
    model_display_name = MODEL_DISPLAY_NAME
    , args = CMDARGS
    , environment_variables = { 'MY_KEY_run'.lower(): 'MY_VALUE_run' }
    , replica_count = 1
    , service_account = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com"
)

In [ ]:
#dir(model)
print(model.name)
print(model.display_name)

In [ ]:
#####################################################################
#
# deploy the model to an endpoint
#
#####################################################################

In [ ]:
#-------------------------------------------------------
# JOB SPECIFIC DETAILS
#-------------------------------------------------------

# exercise details
USE_CASE = "customTrained-prebuiltContainer-2"

# endpoint details
resource_type = "endpt"
resource_name = f"{resource_type}-{USE_CASE}"
DEPLOY_COMPUTE = 'n1-standard-4'

In [ ]:
# create an endpoint
endpoint = aiplatform.Endpoint.create(
    display_name = resource_name
    , project = PROJECT_ID
    , location = REGION
    , labels = {"env" : "prod"}    
)
print(f"Endpoint Created: {endpoint.resource_name}")

In [ ]:
# deploy a model
endpoint.deploy(
    model = model,
    deployed_model_display_name = model.display_name,
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

In [ ]:
#####################################################################
#
# test the prediction server
#
#####################################################################

In [ ]:
# copy data from GCS
TP = os.path.join(GCS_DATA_SOURCE_BUCKET_PATH, TRAIN_DS)
! gsutil cp {TP} .

In [ ]:
# read in data
df = pd.read_csv(TP, nrows=10)
labels = df.pop("label").tolist()
data = df.values.tolist()

In [ ]:
# test prediction server
# send to endpoint
prediction = endpoint.predict(data)
prediction